# Import Packages

In [1]:

# sfsdds
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scrapper.scrape_games import scrape_games
from scrapper.scrape_games import match_players_to_linestar
from utils.data_loader import load_linestar_data

pd.set_option('display.max_rows', None)


# optimizer
from misc.loader import load_dk_contest, get_player_data, parse_lineup
from pydfs_lineup_optimizer import Player,get_optimizer, Site, Sport, CSVLineupExporter, RandomFantasyPointsStrategy, AfterEachExposureStrategy, TeamStack, PlayersGroup, Stack
import re



/Users/kgedin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Functions

In [2]:
# Functions


# Post Processing Optimizer results when contest data is available
def evaluate_lineup(lineup, results_lookup):
    """Return a DataFrame with proj/actual/ownership + lineup totals."""
    rows = []
    for p in lineup.players:
        meta = results_lookup.get(p.full_name, {})
        rows.append({
            "Player": p.full_name,
            "Pos": "/".join(p.positions),
            "Team": p.team,
            "Salary": p.salary,
            "Proj_FPTS": p.fppg,                 # or p.projection, depending on lib
            "Actual_FPTS": meta.get("actual_fpts"),
            "%Drafted": meta.get("ownership"),
        })

    df = pd.DataFrame(rows)
    totals = {
        "Proj_Total": lineup.fantasy_points_projection,
        "Actual_Total": df["Actual_FPTS"].sum(skipna=True),
        "%Own": df["%Drafted"].sum(skipna=True),
    }
    return df, totals



def extract_name(s):
    """
    Turn 'Keyonte George(1408151)' -> 'Keyonte George'
    """
    if pd.isna(s):
        return None
    return re.sub(r"\(\d+\)$", "", str(s)).strip()

def lineup_results(row):
    names = [extract_name(row[c]) for c in pos_cols]
    stats = player_stats.reindex(names)  # allow missing players without errors

    actual_total = stats["Actual"].sum(min_count=1)   # total actual FPTS
    total_own    = stats["Own"].sum(min_count=1)      # sum of %Drafted (or change to .mean())

    return pd.Series({
        "Actual_FP": actual_total,
        "Total_Own": total_own,
    })


# Load Data

In [3]:
slate_date = "2025-11-18"

In [4]:
pbp_features = scrape_games(slate_date, force_rescrape=False)
linestar_df = load_linestar_data(slate_date, normalize=False)

2025-11-22 08:34:37,477 - scrapper.scrape_games - INFO - Loading existing PBP features for 2025-11-18


In [5]:
# ---- 1) Try to load contest data for this slate ----
try:
    dk_df = load_dk_contest(slate_date)
    players_df = get_player_data(dk_df)      # the table you showed
except FileNotFoundError:
    dk_df = None
    players_df = None

has_results = players_df is not None and not players_df.empty

# If we have contest data, normalize column names & build lookup
if has_results:
    results_df = (
        players_df
        .rename(columns={
            "Player": "FullName",     # to match optimizer Player.full_name
            "%Drafted": "ownership",
            "FPTS": "actual_fpts",
        })
    )

    results_lookup = (
        results_df
        .set_index("FullName")[["ownership", "actual_fpts"]]
        .to_dict("index")
    )
else:
    results_lookup = {}


In [7]:
# columns that contain players
pos_cols = ["PG", "SG", "SF", "PF", "C", "G", "F", "UTIL"]

# lookup from player name -> ownership + actual FPTS
player_stats = (
    players_df
    .rename(columns={"%Drafted": "Own", "FPTS": "Actual"})
    .set_index("Player")[["Own", "Actual"]]
)

#linestar_df.columns


# Process Data

### 1. PBP 

In [8]:
# format pbp table


# get team and salary info
aligned = match_players_to_linestar(pbp_features[['Player']], linestar_df['Name'].tolist())
pbp_features = pbp_features.merge(aligned, on='Player', how='left')

#add team and salary info
merge_cols = linestar_df[['Name', 'Team', 'Salary']]
pbp_features = pbp_features.merge(merge_cols, left_on='Player', right_on='Name', how='left')
pbp_features = pbp_features.drop(columns=['Name'])

#Removing players with NaN salary
pbp_features = pbp_features[pbp_features['Salary'].notna()] 

# Removing players who played less than 5 minutes
pbp_features = pbp_features[pbp_features['Minutes'] >= 10]

# dropping momentum colums
pbp_features = pbp_features.drop(columns=['Momentum'])
 
pbp_features.sort_values(by='ClutchFP', ascending=False)

,Player,FPM,Usage,FP,Minutes,ClutchFP,ClutchRatio,TouchesPerMin,ThreeRate,ScoringFrequency,ScoringConsistency,DominantQuarter,LateGameEmphasis,Substitutions,StarterOrStuffer,Team,Salary
6,Ausar Thompson,0.687500,0.250000,16.50,24.0,8.75,0.530303,0.375000,0.000000,0.125000,0.076864,1,0.333333,5.0,0,DET,6400.0
62,Jalen Duren,1.500000,1.000000,43.50,29.0,8.75,0.201149,0.896552,0.000000,0.551724,0.218627,4,0.500000,3.0,1,DET,8000.0
33,Derrick White,1.053030,0.696970,34.75,33.0,8.50,0.244604,0.636364,0.777778,0.181818,0.192439,4,0.500000,2.0,1,BOS,7200.0
19,Cedric Coward,1.355769,0.884615,35.25,26.0,7.50,0.212766,0.615385,0.285714,0.269231,0.148154,3,0.285714,4.0,0,MEM,6300.0
139,Stephen Curry,1.713235,1.411765,58.25,34.0,7.50,0.128755,0.970588,0.538462,0.441176,0.225573,2,0.133333,3.0,0,GSW,9300.0
149,Vince Williams Jr.,1.291667,0.800000,38.75,30.0,7.50,0.193548,0.666667,0.444444,0.266667,0.154699,3,0.250000,4.0,0,MEM,4000.0
152,Wendell Carter Jr.,1.057143,0.571429,37.00,35.0,5.25,0.141892,0.485714,0.500000,0.314286,0.173610,4,0.454545,3.0,0,ORL,5300.0
73,Jaylen Brown,1.234375,1.281250,39.50,32.0,5.00,0.126582,1.031250,0.333333,0.531250,0.209440,3,0.176471,3.0,1,BOS,8800.0
52,Harrison Barnes,1.179688,0.812500,37.75,32.0,4.00,0.105960,0.468750,1.000000,0.312500,0.127976,2,0.400000,3.0,0,SAS,5100.0
112,Nic Claxton,0.858333,0.433333,25.75,30.0,3.50,0.135922,0.666667,0.000000,0.200000,0.146904,2,0.166667,3.0,1,BKN,6400.0


### 2. Player Pool

In [10]:
# format player pool

# Convert linestar_df to salary_df-style structure
lineup_as_salary = pd.DataFrame({
    "Position": linestar_df["Position"],
    "Name + ID": linestar_df["Name"] + " (" + linestar_df["DFS_ID"].astype(str) + ")",
    "Name": linestar_df["Name"],
    "ID": linestar_df["DFS_ID"],
    # simple stand-in: primary position + /UTIL
    "Roster Position": linestar_df["Position"] + "/UTIL",
    # same salary as in linestar_df
    "Salary": linestar_df["Salary"],
    # simple game info: "TEAM VersusStr"  e.g. "OKC vs GSW"
    "Game Info": linestar_df["Team"] + " " + linestar_df["VersusStr"],
    "TeamAbbrev": linestar_df["Team"],
    # use Projected values as AvgPointsPerGame
    "AvgPointsPerGame": linestar_df["Projected"],
    "fppg_floor": linestar_df["Floor"],
    "fppg_ceiling": linestar_df["Ceiling"],
})[[
    "Position",
    "Name + ID",
    "Name",
    "ID",
    "Roster Position",
    "Salary",
    "Game Info",
    "TeamAbbrev",
    "AvgPointsPerGame",
    "fppg_floor",
    "fppg_ceiling"
]]
lineup_as_salary['max_exposure']=100
lineup_as_salary['min_exposure']=0
lineup_as_salary['max_deviation']=0
lineup_as_salary['min_deviation']=0
lineup_as_salary = lineup_as_salary[lineup_as_salary['AvgPointsPerGame'] >= 1]

lineup_as_salary.to_csv('players.csv', index = False)
lineup_as_salary.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,fppg_floor,fppg_ceiling,max_exposure,min_exposure,max_deviation,min_deviation
1,PG,Cade Cunningham (1232182),Cade Cunningham,1232182,PG/UTIL,11000,DET @ATL,DET,44.55,39.82,68.90,100,0,0,0
2,PF,Jalen Johnson (1230356),Jalen Johnson,1230356,PF/UTIL,9500,ATL vs DET,ATL,48.63,26.12,60.45,100,0,0,0
3,PG,Stephen Curry (338365),Stephen Curry,338365,PG/UTIL,9300,GSW @ORL,GSW,42.79,23.33,57.96,100,0,0,0
6,SF,Jaylen Brown (883436),Jaylen Brown,883436,SF/UTIL,8800,BOS @BKN,BOS,39.33,32.39,53.52,100,0,0,0
7,SF,Franz Wagner (1178131),Franz Wagner,1178131,SF/UTIL,8700,ORL vs GSW,ORL,43.02,33.22,46.41,100,0,0,0


# Optimizer

### 1. setup

In [11]:
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.BASKETBALL)
optimizer.load_players_from_csv('players.csv')

In [12]:
pp_list = [
    'Markkanen', 'Sharpe', 'Giddey', 'Okongwu', 'Clingan', 'Grant',
    'Avdija', 'George', 'Gillespie', 'Silva', 'Black', 'Goodwin',
    'Brooks', 'Queen', 'Johnson', 'Buzelis', 'Camara', 'Sheppard',
    'Gafford', 'Sengun'
]

pp_list = [
    'Johnson', 'Fox', 'Porter Jr.', 'White', 'Okongwu', 'Wagner',
    'Duren', 'Curry', 'Cunningham', 'Brown', 'Daniels', 'Jackson Jr.',
    'Butler', 'Claxton', 'Kornet', 'Aldama', 'Vassell', 'Champagnie',
    'Stewart', 'Bane'
]

# Assuming 'optimizer' and 'optimizer.player_pool' are already defined and populated.

# Define the new deviation values you want to apply to these players
new_min_deviation = 0.3  # Example value
new_max_deviation = 0.6  # Example value

# Loop through the list of player names and update their deviations
for player_name in pp_list:
    # Retrieve the player object from the optimizer's player pool
    player = optimizer.player_pool.get_player_by_name(player_name)
    
    # Check if the player was found before trying to set attributes
    if player:
        player.min_deviation = new_min_deviation
        player.max_deviation = new_max_deviation
        # Optional: you could print a confirmation
        # print(f"Updated deviation for {player_name}")
    else:
        # Handle cases where a name might not be in the pool
        print(f"Warning: Player '{player_name}' not found in player pool.")

# The rest of your existing code still applies globally or to specific individuals as needed:
optimizer.set_fantasy_points_strategy(RandomFantasyPointsStrategy(max_deviation=0.2))

# You can still set specific exceptions for other players outside the list:
#harden = optimizer.player_pool.get_player_by_name('Harden')
#if harden:
#    harden.min_deviation = 0.3
#    harden.max_deviation = 0.6

#westbrook = optimizer.player_pool.get_player_by_name('Westbrook')
#if westbrook:
#    westbrook.min_deviation = 0
#    westbrook.max_deviation = 0


### 2. run

In [13]:
#has_results = True

for lineup in optimizer.optimize(n=3):#, max_exposure=0.5):
    print(lineup)                         # existing printout

    if has_results:
        eval_df, totals = evaluate_lineup(lineup, results_lookup)
        #display(eval_df)                  # per-player view with proj / actual / %Drafted
        #print("Projected FP:", totals["Proj_Total"])
        print("Actual FP:   ", totals["Actual_Total"])
        print("%Own:", totals["%Own"])
        print('\n\n\n')


 1. PG      Dyson Daniels                 PG    ATL                     34.39(54.978)  7500.0$   
 2. SG      Drake Powell                  SF/SG BKN                     13.3(11.932)   3600.0$   
 3. SF      Devin Vassell                 SF    SAS                     31.69(49.494)  6500.0$   
 4. PF      Isaiah Stewart                C/PF  DET                     27.69(43.24)   5600.0$   
 5. C       Nic Claxton                   C     BKN                     31.93(43.813)  6400.0$   
 6. G       Stephen Curry                 PG    GSW                     42.79(58.491)  9300.0$   
 7. F       Ziaire Williams               PF/SF BKN                     18.95(21.525)  3900.0$   
 8. UTIL    Onyeka Okongwu                C     ATL                     37.33(56.581)  7200.0$   

Fantasy Points 238.07(340.05)
Salary 50000.00

Actual FP:    243.25
%Own: 94.60999999999999




 1. PG      Dyson Daniels                 PG    ATL                     34.39(52.241)  7500.0$   
 2. SG      Desmond B

# Backup

In [ ]:
# Optimizer settings

# number of lineups
n = 10 

# max repeating player
max_repeat= 5


# minimum salary for lineup
optimizer.set_min_salary_cap(49000)


# max/min teams
min_teams=4
max_teams=6

# team exposures
#optimizer.set_teams_max_exposures(0.5)  # Set 0.5 exposure for all teams
#optimizer.set_teams_max_exposures(0.5, exposures_by_team={'NYY': 0.8})  # Set 0.5 exposure for all teams except NYY and 0.8 exposure for NYY
#optimizer.set_teams_max_exposures(exposures_by_team={'NYY': 0.8})  # Set 0.5 exposure only for NYY
#optimizer.set_teams_max_exposures(exposures_by_team={'NYY': 0.5, 'NYM': 0.5}, exposure_strategy=AfterEachExposureStrategy)  # Use another exposure strategy



# STACKING


# team stacking

optimizer.add_stack(TeamStack(3))  # stack 3 players
optimizer.add_stack(TeamStack(3, for_teams=['NE', 'BAL', 'KC']))  # stack 3 players from any of specified teams
optimizer.add_stack(TeamStack(3, for_positions=['QB', 'WR', 'TE']))  # stack 3 players with any of specified positions
optimizer.add_stack(TeamStack(3, spacing=2))  # stack 3 players close to each other in range of 2 spots.
optimizer.add_stack(TeamStack(3, max_exposure=0.5))  # stack 3 players from same team with 0.5 exposure for all team stacks
optimizer.add_stack(TeamStack(3, max_exposure=0.5, max_exposure_per_team={'MIA': 0.6}))  # stack 3 players from same team with 0.5 exposure for all team stacks and 0.6 exposure for MIA


# Position Stack

optimizer.add_stack(GameStack(3))  # stack 3 players from the same game
optimizer.add_stack(GameStack(5, min_from_team=2))  # stack 5 players from the same game, 3 from one team and 2 from another



# custom group stack
rodgers_adams_group = PlayersGroup(optimizer.player_pool.get_players('Aaron Rodgers', 'Davante Adams'), max_exposure=0.5)
brees_thomas_group = PlayersGroup(optimizer.player_pool.get_players('Drew Brees', 'Michael Thomas'), max_exposure=0.5)
optimizer.add_stack(Stack([rodgers_adams_group, brees_thomas_group]))



# additional stack




NameError: name 'TeamStack' is not defined

In [ ]:


optimizer.set_total_teams(min_teams=min_teams, max_teams=max_teams)
optimizer.set_max_repeating_players(max_repeat)
lineups = list(optimizer.optimize(n))
optimizer.export('result.csv')

In [ ]:
lu = pd.read_csv('result.csv', index_col = False)
lu.head()

,PG,SG,SF,PF,C,G,F,UTIL,Budget,FPPG
0,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Tyrese Maxey(1186160),Kyle Filipowski(1403609),Jusuf Nurkic(830642),50000.0,270.74
1,Keyonte George(1408151),Ace Bailey(1519363),Michael Porter Jr.(1061724),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),OG Anunoby(900535),50000.0,270.01
2,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Neemias Queta(1142902),50000.0,269.31
3,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Santi Aldama(1177024),50000.0,269.25
4,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Trendon Watford(1176529),50000.0,268.54


In [ ]:
lu

In [ ]:





# add columns to your lineup df
lu[["Actual_FP", "Total_Own"]] = lu.apply(lineup_results, axis=1)

lu.head()


,PG,SG,SF,PF,C,G,F,UTIL,Budget,FPPG,Actual_FP,Total_Own
0,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Tyrese Maxey(1186160),Kyle Filipowski(1403609),Jusuf Nurkic(830642),50000.0,270.74,222.50,211.01
1,Keyonte George(1408151),Ace Bailey(1519363),Michael Porter Jr.(1061724),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),OG Anunoby(900535),50000.0,270.01,253.00,211.33
2,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Neemias Queta(1142902),50000.0,269.31,231.75,183.79
3,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Andre Drummond(614746),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Santi Aldama(1177024),50000.0,269.25,241.00,174.72
4,Keyonte George(1408151),Payton Pritchard(945610),OG Anunoby(900535),Collin Murray-Boyles(1465110),Jusuf Nurkic(830642),Shai Gilgeous-Alexander(1067856),Kyle Filipowski(1403609),Trendon Watford(1176529),50000.0,268.54,216.25,195.42
